In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import \
    BaggingClassifier, VotingClassifier, \
    AdaBoostClassifier, StackingClassifier, \
    RandomForestClassifier

from sklearn.metrics import accuracy_score, classification_report

import xgboost as xgb

C:\Users\a1056968\Anaconda3\envs\machine_learning\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


# Comparison between Ensemble Algorithms

## Adult dataset

**Prediction task** is to determine whether a person makes over 50K a year.

**Objective**: Сравнение на различните ансамбли.
- Какво е значението на типа ансамбъл?
- Каква оптимизация на хиперпараметри (ХП) трябва?
- Кои от ХП да използваме при оптимизация (пространство на ХП)?
- Как може да ги сравним:
    - колко бързо работят
    - възможност за паралелно изпълнение (на процесор)
- Колко лесно е да интерпретираме, разбрем резултатите (explainability)?
  
    




Data from [here](https://archive.ics.uci.edu/ml/datasets/adult).

### Process the data

In [3]:
income_data = pd.read_csv("../datasets/adult/adult.data", header=None)
income_data.columns = [
    "age",
    "workclass",
    "final_weight",
    "education",
    "education-num",
    "marital-status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "capital_gain",
    "capital_loss",
    "hours_per_week",
    "native_country",
    "income_class",
]

income_target = income_data.income_class
income_target = income_target.str.strip()

income_attributes = income_data.drop(columns="income_class")
income_attributes = pd.get_dummies(income_attributes, drop_first=True)
scaler = MinMaxScaler()
income_attributes = scaler.fit_transform(income_attributes)

income_attributes_train, income_attributes_test, \
income_target_train, income_target_test, \
= train_test_split(income_attributes, income_target, train_size=0.8, test_size=0.2)

print("Shape of the train and test sets for the features and target:")
for group in [income_attributes_train, income_attributes_test, income_target_train, income_target_test]:
    print(group.shape)

Shape of the train and test sets for the features and target:
(26048, 100)
(6513, 100)
(26048,)
(6513,)


In [8]:
pd.DataFrame

array([[0.30136986, 0.0443019 , 0.8       , ..., 1.        , 0.        ,
        0.        ],
       [0.45205479, 0.0482376 , 0.8       , ..., 1.        , 0.        ,
        0.        ],
       [0.28767123, 0.13811345, 0.53333333, ..., 1.        , 0.        ,
        0.        ],
       ...,
       [0.56164384, 0.09482688, 0.53333333, ..., 1.        , 0.        ,
        0.        ],
       [0.06849315, 0.12849934, 0.53333333, ..., 1.        , 0.        ,
        0.        ],
       [0.47945205, 0.18720338, 0.53333333, ..., 1.        , 0.        ,
        0.        ]])

In [4]:
print("Proportion of target classses:")
print(
    "Training set:\t",
    income_target_train.value_counts(normalize=True).values.round(2)
)

print(
    "Test set:\t",
    income_target_test.value_counts(normalize=True).values.round(2)
)

Proportion of target classses:
Training set:	 [0.76 0.24]
Test set:	 [0.76 0.24]


In [5]:
X_train, y_train = income_attributes_train, income_target_train
X_test, y_test = income_attributes_test, income_target_test

In [6]:
def print_model_score(model, on_training=False):
    print("On test data:")
    print(f"Default estimator score: {model.score(X_test, y_test):.2f}")
    print(f"Accuracy:\t\t {accuracy_score(y_test, model.predict(X_test)):.2f}")
    
    if on_training:
        print("On training data:")
        print(f"Default estimator score: {model.score(X_train, y_train):.2f}")
        print(f"Accuracy:\t\t {accuracy_score(y_train, model.predict(X_train)):.2f}")

### Classifier estimators

#### Decision tree

In [15]:
tree_depth1 = DecisionTreeClassifier(max_depth=1)
tree_depth1.fit(X_train, y_train)

tree_depth2 = DecisionTreeClassifier(max_depth=2)
tree_depth2.fit(X_train, y_train)

tree_depth4 = DecisionTreeClassifier(max_depth=4)
tree_depth4.fit(X_train, y_train)

tree_depth8 = DecisionTreeClassifier(max_depth=8)
tree_depth8.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=8)

In [16]:
print("Decision tree scores")

print("Depth 1")
print_model_score(tree_depth1)

print("\nDepth 2")
print_model_score(tree_depth2)

print("\nDepth 4")
print_model_score(tree_depth4)

print("\nDepth 8")
print_model_score(tree_depth8)

Decision tree scores
Depth 1
On test data:
Default estimator score: 0.76
Accuracy:		 0.76

Depth 2
On test data:
Default estimator score: 0.83
Accuracy:		 0.83

Depth 4
On test data:
Default estimator score: 0.84
Accuracy:		 0.84

Depth 8
On test data:
Default estimator score: 0.85
Accuracy:		 0.85


In [18]:
print("Classification reports\n")

all_trees = [tree_depth1, tree_depth2, tree_depth4, tree_depth8]
titles = [f"Tree, max depth={x}" for x in list("1248")]

for a_tree, title in zip(all_trees, titles):
    report = classification_report(
        y_test, 
        a_tree.predict(X_test),
        zero_division=False
    )
    print(title, "\n", report, "\n")

Classification reports

Tree, max depth=1 
               precision    recall  f1-score   support

       <=50K       0.76      1.00      0.87      4965
        >50K       0.00      0.00      0.00      1548

    accuracy                           0.76      6513
   macro avg       0.38      0.50      0.43      6513
weighted avg       0.58      0.76      0.66      6513
 

Tree, max depth=2 
               precision    recall  f1-score   support

       <=50K       0.85      0.95      0.89      4965
        >50K       0.72      0.46      0.56      1548

    accuracy                           0.83      6513
   macro avg       0.79      0.70      0.73      6513
weighted avg       0.82      0.83      0.81      6513
 

Tree, max depth=4 
               precision    recall  f1-score   support

       <=50K       0.86      0.94      0.90      4965
        >50K       0.74      0.51      0.61      1548

    accuracy                           0.84      6513
   macro avg       0.80      0.73      0

#### Random forest

In [9]:
forest_5 = RandomForestClassifier(n_estimators=5)
forest_5.fit(X_train, y_train)

forest_10 = RandomForestClassifier(n_estimators=10)
forest_10.fit(X_train, y_train)

forest_30 = RandomForestClassifier(n_estimators=30)
forest_30.fit(X_train, y_train)

RandomForestClassifier(n_estimators=30)

The random tree score is the mean accuracy on the given test data and labels.

In [10]:
all_forests = [forest_5, forest_10, forest_30, ]
titles = [f"Random forest, number of estimators {x}: " for x in [5, 10, 30]]

print("Score = mean accuracy\n")
for a_forest, title in zip(all_forests, titles):
    score = a_forest.score(X_test, y_test)
    print(f"{title}{score:.3f}")

Score = mean accuracy

Random forest, number of estimators 5: 0.839
Random forest, number of estimators 10: 0.846
Random forest, number of estimators 30: 0.852


### Comparisons

#### Bagging vs Voting

In [27]:
bag_2 = BaggingClassifier(tree_depth2, n_estimators=20)
bag_2.fit(X_train, y_train)

bag_4 = BaggingClassifier(tree_depth4, n_estimators=20)
bag_4.fit(X_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=4),
                  n_estimators=20)

In [28]:
vote_2 = VotingClassifier(
    estimators=[(f"clf{i}", tree_depth2) for i in range(20)],
    voting="hard",
)
vote_2.fit(X_train, y_train)


vote_4 = VotingClassifier(
    estimators=[(f"clf{i}", tree_depth4) for i in range(20)],
    voting="hard",
)
vote_4.fit(X_train, y_train)

VotingClassifier(estimators=[('clf0', DecisionTreeClassifier(max_depth=4)),
                             ('clf1', DecisionTreeClassifier(max_depth=4)),
                             ('clf2', DecisionTreeClassifier(max_depth=4)),
                             ('clf3', DecisionTreeClassifier(max_depth=4)),
                             ('clf4', DecisionTreeClassifier(max_depth=4)),
                             ('clf5', DecisionTreeClassifier(max_depth=4)),
                             ('clf6', DecisionTreeClassifier(max_depth=4)),
                             ('clf7...
                             ('clf12', DecisionTreeClassifier(max_depth=4)),
                             ('clf13', DecisionTreeClassifier(max_depth=4)),
                             ('clf14', DecisionTreeClassifier(max_depth=4)),
                             ('clf15', DecisionTreeClassifier(max_depth=4)),
                             ('clf16', DecisionTreeClassifier(max_depth=4)),
                             ('clf17', DecisionTreeClassifier(max_depth=4)),
                             ('clf18', DecisionTreeClassifier(max_depth=4)),
                             ('clf19', DecisionTreeClassifier(max_depth=4))])

In [30]:
all_bags = [bag_2, bag_4]
all_votes = [vote_2, vote_4]
titles = [f"Estimator: decision tree with depth {x}: " for x in [2, 4]]

print("Scores Bagging vs Voting\n")
for bag, vote, title in zip(all_bags, all_votes, titles):
    bag_score = bag.score(X_test, y_test)
    vote_score = vote.score(X_test, y_test)
    
    print(title)
    print(f"Bagging: {bag_score:.3f}")
    print(f"Votting: {vote_score:.3f}")    

Scores Bagging vs Voting

Estimator: decision tree with depth 2: 
Bagging: 0.829
Votting: 0.829
Estimator: decision tree with depth 4: 
Bagging: 0.842
Votting: 0.841


#### AdaBoost vs StackingClassifier

In [14]:
ada_2 = AdaBoostClassifier(base_estimator=tree_depth2, n_estimators=)

TypeError: __init__() got an unexpected keyword argument 'base_estimator'